In [46]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [47]:
ticker_prices_filename = 'ticker_prices.xlsx'
ticker_prices_excel_book = pd.read_excel(io=ticker_prices_filename, header=[0,1], index_col=0, sheet_name=None)

In [49]:
period_column_name = 'period'
Close_column_name = 'Close'
price_column_name = 'price'
Volume_column_name = 'Volume'
volume_column_name = 'volume'
first_index = -11
market_suffix = '.JK'

output_sheet_list = []

for sheet_name in ticker_prices_excel_book.keys():
    data_frame = ticker_prices_excel_book[sheet_name].copy()
    number_of_rows = len(data_frame.index)
    data_frame[period_column_name] = list(range(first_index, first_index+number_of_rows))
    data_frame = data_frame.copy().set_index(period_column_name)
    data_frame = data_frame.copy().rename(columns={Close_column_name: price_column_name, Volume_column_name: volume_column_name}, level=0)
    data_frame.columns = pd.MultiIndex.from_tuples([(i[0], i[1].replace(market_suffix, '')) for i in data_frame.columns])
    
    output_sheet_list.append({
        'sheet_name': sheet_name,
        'data_frame': data_frame,
    })


In [50]:
with pd.ExcelWriter('ticker_prices_transformed.xlsx') as excel_writer:
    for i in output_sheet_list:
        sheet_name = i['sheet_name']
        df = i['data_frame']
        df.index = df.index.map(str)
        df.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)